# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
data = pd.read_csv('/datasets/users_behavior.csv')
display(data.head())
data.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Вывод:

Мне дана таблица на `3214` объектов и `5` признаков.

Таблица без пропусков, типы данных соответствуют.

Целевой признак - `is_ultra`

Целевой признак категориальный, категорий всего 2, тоесть это бинарная классификация.

`calls, minutes, messages, mb_used` - признаки

## Разбейте данные на выборки

In [2]:
data_train, data_valid, data_test = np.split(data, [int(.6*len(data)), int(.8*len(data))])

print("Размер обучающей выборки:", data_train.shape)
print("Размер валидационной выборки:", data_valid.shape)
print("Размер тестовой выборки:", data_test.shape)

Размер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выборки: (643, 5)


In [3]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra'] 

## Вывод:

В данном шаге я разбил данные на 3 выборки размерами `60%, 20%, 20%`.

Далее из всех 3 выборок выделяю признаки и записываю во `features_` и целевой признак и записываю его в `target_`.

## Исследуйте модели

In [4]:
best_depth = 0
max_result = 0
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > max_result:
        best_depth = depth
        max_result = result
print('max_depth:', best_depth)
print('accuracy:', max_result) 

max_depth: 9
accuracy: 0.8133748055987559


Я начал с обучения модели дерева решений.

Наибольшее значение `accuracy` стал искать, меняя параметр `max_depth` в диапазоне `1-10`.

Установил: если значение параметра меньше 9, то модель недообучается и если больше 9, то переобучается.

In [5]:
best_est = 0
max_result = 0
for est in range(70, 81):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_forest.fit(features_train, target_train)
    predictions_valid = model_forest.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > max_result:
        max_result = result
        best_est = est
print('n_estimators:', best_est)
print('accuracy:', max_result) 

n_estimators: 73
accuracy: 0.8133748055987559


Далее идёт модель случайного леса.

Изначально параметр `n_estimators` я искал в диапазоне от `1 до 500` и нашёл максимальное `accuracy` при `n_estimators=73`, далее для уменьшения времени на вычисления я сузил границы от `70 до 80`.

Значение `accuracy` полностью соответствует значению модели дерева решений, но по скорости сильно ей уступает.

In [6]:
model_logreg = LogisticRegression(random_state=12345)
model_logreg.fit(features_train, target_train)
predictions_valid = model_logreg.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('accuracy:', result)

accuracy: 0.7527216174183515


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Модель номер 3 - логистическая регрессия.

Значение accuracy получилось минимальное среди трёх моделей, но скорость работы выше всего.

## Вывод:

После обучения трёх моделей видно, что наиболее точны модель дерева решений и модель случайного леса, а по скорости работы модели распределились в следующем порядке: логистическая регрессия, дерево решений, случайный лес.

Из всего этого делаю вывод, что наиболее удачная - модель дерева решений.

## Проверьте модель на тестовой выборке

In [7]:
model_tree = DecisionTreeClassifier(random_state=12345, max_depth=9)
model_tree.fit(features_train, target_train)
predictions_test = model_tree.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('accuracy:', result)

accuracy: 0.7993779160186625


In [8]:
model_forest = RandomForestClassifier(random_state=12345, n_estimators=73)
model_forest.fit(features_train, target_train)
predictions_test = model_forest.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('accuracy:', result)

accuracy: 0.8040435458786936


In [9]:
predictions_test = model_logreg.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('accuracy:', result)

accuracy: 0.7293934681181959


## Вывод:

По всей видимости, в предыдущем шаге я поспешил с выводами и, не смотря на меньшую скорость работы, в тестовых испытаниях наибольшую точность показала модель случайного леса, тоесть теперь очевидно, что лучшая модель для точности - случайный лес, для скорости и точности - дерево решений.

## (бонус) Проверьте модели на адекватность

In [11]:
model_dummy = DummyClassifier(random_state=12345, strategy='uniform')
model_dummy.fit(features_train, target_train)

predictions_valid = model_dummy.predict(features_valid)
result_valid = accuracy_score(target_valid, predictions_valid)
print('accuracy valid:', result_valid)

predictions_test = model_dummy.predict(features_test)
result_test = accuracy_score(target_test, predictions_test)
print('accuracy test:', result_test)

accuracy valid: 0.49455676516329705
accuracy test: 0.5116640746500778


Для сравнения с моделями я выбрал тривиальную модель `uniform`.

Получил сильно меньшие, чем у моих моделей значения и для валидационной выборки и для тестовой выборки.

Это говорит, что мои модели прошли проверку на адекватность, ведь значения получены большие, чем от тривиальной.

## Вывод:

В общем делаю следующий вывод, что по условию требовалась модель с точностью ответов не ниже 0.75, следовательно прошли модель дерева решений и модель случайного леса.

Следующее условие - модель должна быть адекватна, и с этим условием справились уже все 3 модели.

Accuracy на тестовой выборке показала для всех трёх моделей снижение качества предсказаний в сравнении с валидационной выборкой.

В итоге получаем 2 хорошие модели из трёх, а дальше уже нужно плясать от требований заказчиков, если нужна скорость, то это дерево решений, если важна максимальная точность - это случайный лес.